# Skin cancer detection

Dataset taken from Kaggle at [Skin Cancer ISIC](https://www.kaggle.com/datasets/nodoubttome/skin-cancer9-classesisic).

In [1]:
from keras_preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import numpy as np
import os, shutil
import pandas as pd

import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
sns.set_theme(style="whitegrid")

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_digits, load_sample_images
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer

import tensorflow as tf
import keras
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier

# fix random seed for reproducibility
seed = 42
tf.random.set_seed(seed)

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Directory path
train_data_dir = 'dataskin/Train'
test_data_dir = 'dataskin/Test'

# Get all the data in the directory data/validation (132 images), and reshape them
test_generator = ImageDataGenerator().flow_from_directory(
        test_data_dir, batch_size=118)

# Get all the data in the directory data/train (790 images), and reshape them
train_generator = ImageDataGenerator().flow_from_directory(
        train_data_dir, batch_size=2239)

# Create the datasets
train_images_fs, train_labels = next(train_generator)
test_images_fs, test_labels = next(test_generator)
#val_images, val_labels = next(val_generator)

Found 118 images belonging to 9 classes.
Found 2239 images belonging to 9 classes.


In [ ]:
train_images_fs

In [ ]:
# Preview an image
array_to_img(train_images_fs[0])

In [ ]:
# Preview an image
array_to_img(test_images_fs[0])

In [ ]:
train_labels.shape

In [ ]:
test_labels.shape

In [ ]:
test_images_fs.shape

In [ ]:
train_images_fs.shape

In [ ]:
train_labels

In [ ]:
type(train_labels)

In [ ]:
type(train_images_fs)

In [ ]:
dataframe=pd.DataFrame(train_labels, columns=['actinic_keratosis', 'basal_cell_carcinoma',\
            'dermatofibroma', 'melanoma', 'nevus', 'pigmented_benign_keratosis',\
            'seborrheic_keratosis', 'squamous_cell_carcinoma', 'vascular_lesion'])

In [ ]:
dataframe

In [ ]:
#dataframe.hist()

In [ ]:
sums=dataframe.sum(axis=0)

In [ ]:
sums=pd.DataFrame(sums)

In [ ]:
sums

In [ ]:
sums['%'] = ((sums[0] / 2239*100).round(2).astype(str) + '%')

In [ ]:
sums

In [ ]:
#Change this into seaborn to make it even cooler
sums.plot(kind='bar');

In [ ]:
train_generator.class_indices 

Defining a function to visualize the results.

In [ ]:
"""def visualize_training_results(results):
    history = results.history
    plt.figure()
    plt.plot(history['val_loss'])
    plt.plot(history['loss'])
    plt.legend(['val_loss', 'loss'])
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.show()
    
    plt.figure()
    plt.plot(history['val_accuracy'])
    plt.plot(history['accuracy'])
    plt.legend(['val_accuracy', 'accuracy'])
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.show()"""

Look at lab on https://github.com/learn-co-curriculum/dsc-image-classification-with-mlps-lab/tree/solution
might need to do some one hot encoding.
Also, just like it shows there, as a naive model maybe decrease the number of layers and keep epochs at like 5.
If results are the same, try to understand why the loss still showes such high peaks.And why the accuracy drops like that.
Might also be worth doing CNN from the start with 2D layers instead of this basic NN.

Let us forget about all of this and try with the 2D actual convolutional neural network for images. The perfmormance there might be different.

We can also do something else which is group together the types of skin abnormalities that are cancerous and the ones that are not, reducing the classes to 2 and making this a binary classification problem.
I will do that later on. For now let's see what happens with a convolutional neural network, which is what I should be doing anyway.

Here also definitely preview one image per class and explain a bit about each class.

## Preprocessing

In [ ]:
# Directory path
train_data_dir = 'dataskin/Train'
test_data_dir = 'dataskin/Test'
#val_data_dir = 'dataset/validation_set'

# Get all the data in the directory data/validation (118 images), and reshape them
#test_gen= ImageDataGenerator().flow_from_directory(
#        test_data_dir, target_size=(32, 32), batch_size=118)

# Get all the data in the directory data/train (2239 images), and reshape them
train_gen = ImageDataGenerator().flow_from_directory(
        train_data_dir, target_size=(32, 32), batch_size=2239)

# Create the datasets
train_img32, train_lab = next(train_gen)
test_img32, test_lab = next(test_gen)

In [ ]:
# Explore your dataset again
m_train = train_img32.shape[0]
num_px = train_img32.shape[1]
m_test = test_img32.shape[0]

print ("Number of training samples: " + str(m_train))
print ("Number of testing samples: " + str(m_test))
print ("train_images shape: " + str(train_img32.shape))
print ("train_labels shape: " + str(train_lab.shape))
print ("test_images shape: " + str(test_img32.shape))
print ("test_labels shape: " + str(test_lab.shape))

In [ ]:
train_lab.shape[0]

In [ ]:
#train_img = train_img.reshape(train_img.shape[0], -1)
#test_img = test_img.reshape(test_img.shape[0], -1)
#val_img = val_images.reshape(val_images.shape[0], -1)

#print(val_img.shape)

As we can see the RGB values of each pixel are not scaled, we are going to scale their value between 0 and 1 by dividing each pixel value by 225,
and we are also going to reshape the data in a format that can be fed into the model.
We are going to put these two preprocessing steps together with a pipeline.

#### Here I am going to create my first pipeline 
to use to scale and reshape the data once I have loaded it
Or possibly even a function to load it.

to standardize the pixel values:

In [ ]:
def scale_pixels(train_img, test_img, train_lab, test_lab):
    train_img /= 255.
    test_img /= 255.
    return train_img, test_img

In [ ]:
# Instantiate transformer
pix_scaler = FunctionTransformer(scale_pixels)

In [ ]:
"""pix_scaler.fit_transform(train_img32, test_img32, train_lab, test_lab)"""

In [ ]:
#pix_scaler.transform(train_img, test_img)

In [ ]:
#from sklearn.preprocessing import StandardScaler
#StandardScaler().fit_transform(train_img)

In [ ]:
test_img32.shape

In [ ]:
train_img32.shape

In [ ]:
# Instantiate StandardScaler
#scaler = MinMaxScaler()
# Transform the training and test sets
#scaled_data_train = scaler.fit_transform(train_img)
#scaled_data_test = scaler.transform(test_img)

# Convert into a DataFrame
#scaled_df_train = pd.DataFrame(scaled_data_train, columns=X_train.columns)
#scaled_df_train.head()

In [ ]:
#scale_pixels(train_img, test_img)

In [ ]:
#train_img

To change the shape of the labels:

In [ ]:
def reshape (train_lab, test_lab): #(train_img, test_img, train_lab, test_lab):
    m=train_lab.shape[0]
    n=test_lab.shape[0]
    train_lab = np.reshape(train_lab[:,0], (m,1))
    test_lab = np.reshape(test_lab[:,0], (n,1))
    return train_lab, test_lab

In [ ]:
# Instantiate transformer
shaper = FunctionTransformer(reshape)

In [ ]:
print(train_lab.shape)
print(test_lab.shape)

In [ ]:
#train_y, test_y=reshape(train_lab, test_lab)

In [ ]:
"""shaper.fit_transform(train_lab, test_lab)#(train_img, test_img, train_lab, test_lab)"""

In [ ]:
print(train_lab.shape)
print(test_lab.shape)

In [ ]:
train_y = np.reshape(train_lab[:,0], (2239,1))
test_y = np.reshape(test_lab[:,0], (118,1))

In [ ]:
print(train_y.shape)
print(test_y.shape)

In [ ]:
"""ct=make_column_transformer(
    (pix_scaler, ['img_scaled']),
    (shaper, ['labels_reshaped']))"""

In [ ]:
"""ct.fit_transform(train_img, test_img, train_lab, test_lab)"""

In [ ]:
# Build a pipeline with StandardScaler and KNeighborsClassifier
"""scale_pipeline = Pipeline([('pixel scaler', pix_scaler(train_img, test_img)),
        ('changing shape', shaper(train_lab, test_lab))])"""

In [ ]:
"""scale_pipeline.fit_transform(train_img, test_img, train_lab, test_lab)"""

In [ ]:
"""# Fit the training data to pipeline
scaled_pipeline_1.fit(X_train, y_train)

# Print the accuracy on test set
scaled_pipeline_1.score(X_test, y_test)"""

Building two functions that will help us visualize and compare the different models:

In [ ]:
def visualize_training_results(results):
    history = results.history
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,6))
    ax1.plot(history['val_acc'])
    ax1.plot(history['acc'])
    ax1.legend(['val_acc', 'acc'], loc='best')
    ax1.set_title('Accuracy')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Accuracy')
    
    ax2.plot(history['val_loss'])
    ax2.plot(history['loss'])
    ax2.legend(['val_loss', 'loss'], loc='best')
    ax2.set_title('Loss')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Loss')

Creating the empty dataframe where we will store the results:

In [ ]:
# create an Empty DataFrame object
df = pd.DataFrame()

In [ ]:
def model_results(results, model_name, df):
    visualize_training_results(results)
    df1 = pd.DataFrame({'Name': [model_name],
                        'accuracy train': [round(results.history['acc'][-1],5)] ,
                        'accuracy validation': [round(results.history['val_acc'][-1],5 )],
                        'loss train': [round(results.history['loss'][-1],5)],
                        'loss validation': [round(results.history['val_loss'][-1],5)]})
    df=df.append(df1)
    fig = plt.subplots(figsize=(12,6))
    
    plt.subplot(1,2,1)
    x=df['Name']
    y=df['accuracy train']
    plt.plot(x, y, color='r')
    x1=df['Name']
    y1=df['accuracy validation']
    plt.plot(x1, y1, color='g')
    plt.xticks(x, labels=df['Name'], rotation='vertical')
    plt.title('Learning Curve Accuracy')
    plt.legend()
 
    plt.subplot(1,2,2)
    x=df['Name']
    y=df['loss train']
    plt.plot(x, y, color='r')
    x1=df['Name']
    y1=df['loss validation']
    plt.plot(x1, y1, color='g')
    plt.xticks(x, labels=df['Name'], rotation='vertical')
    plt.title('Learning Curve Loss')
    plt.legend()

    plt.show()
    return df

And let us create another function to generate models.
With this function we can set all the parameters that we want, input_shape, activation function, loss, optimizer algorithm, metrics etc.
We will also set most of these parameters as default, except for input shape, to make the execution of the function not too heavy.

In [ ]:
def create_model(input_shape, activation='relu', loss='sparse_categorical_crossentropy', 
                optimizer='rmsprop',metrics=['acc'] , neur1=10, neur2=32, pool=(2,2)):
 # create model
    model = models.Sequential()
    model.add(layers.Conv2D(neur1, (3, 3), activation=activation,
                        input_shape=input_shape))
    model.add(layers.MaxPooling2D(pool))
    
    model.add(layers.Conv2D(neur1, (4, 4), activation=activation))
    model.add(layers.MaxPooling2D(pool))

    model.add(layers.Conv2D(neur2, (3, 3), activation=activation))
    model.add(layers.MaxPooling2D(pool))

    model.add(layers.Flatten())
    model.add(layers.Dense(32, activation=activation))
    model.add(layers.Dense(9, activation='softmax'))

# Compile model
    model.compile(loss=loss,
              optimizer=optimizer,
              metrics=metrics)
    return model

And one more function to fit the model and get the results. Also here we can set number of epochs, batch size and validation split, but there are default parameters too.

In [ ]:
def fit_model(model, train, y, epo=10, batch=10, val_split=0.3):
    results = model.fit(train, y, epochs=epochs, batch_size=batch, validation_split=val_split)
    return results

In [ ]:
#visualize_training_results(history)

**Let me try them**

In [ ]:
mod1=create_model((32,32,3), neurons1=15, neurons2=20 )

In [ ]:
results=fit_model(mod1, train_img32, train_y)

### Trying to pick a Naive model

Trying with a simple model:

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(10, (3, 3), activation='relu',
                        input_shape=(32 ,32,  3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(9, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

And doing only 10 epochs:

In [ ]:
history1 = model.fit(train_img32,
                    train_y,
                    epochs=10,
                    batch_size=10, validation_split=0.3)

Let us visualize the results of this first model:
The bottom two graphs will make sense only later on since we are building a learning curve, to compare the different models' performance.

In [ ]:
df=model_results(history1, '32x32 1st model', df)
df

Another attempt: all the previous parameters but making the last dense layer smaller.

In [ ]:
model2 = models.Sequential()
model2.add(layers.Conv2D(10, (3, 3), activation='relu',
                        input_shape=(32 ,32,  3)))
model2.add(layers.MaxPooling2D((2, 2)))

model2.add(layers.Flatten())
model2.add(layers.Dense(10, activation='relu'))
model2.add(layers.Dense(9, activation='softmax'))

model2.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [ ]:
history3 = model2.fit(train_img32,
                    train_y,
                    epochs=10,
                    batch_size=50, validation_split=0.2)

In [ ]:
df=model_results(history3, '32x32 3rd model, < nrns last layer', df)
df

#### Even less Neurons:

In [ ]:
model3 = models.Sequential()
model3.add(layers.Conv2D(5, (3, 3), activation='relu',
                        input_shape=(32 ,32,  3)))
model3.add(layers.MaxPooling2D((2, 2)))

model3.add(layers.Flatten())
model3.add(layers.Dense(5, activation='relu'))
model3.add(layers.Dense(9, activation='softmax'))

model3.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [ ]:
history4 = model3.fit(train_img32,
                    train_y,
                    epochs=10,
                    batch_size=50, validation_split=0.2)

In [ ]:
df=model_results(history4, '32x32 4th model, < neurons', df)
df

#### Less resolution images.

In [ ]:
# Directory path
train_data_dir = 'dataskin/Train'
#test_data_dir = 'dataskin/Test'

# Get all the data in the directory data/validation (118 images), and reshape them
#test_gen= ImageDataGenerator().flow_from_directory(
#        test_data_dir, target_size=(8, 8), batch_size=118)

# Get all the data in the directory data/train (2239 images), and reshape them
train_gen = ImageDataGenerator().flow_from_directory(
        train_data_dir, target_size=(8, 8), batch_size=2239)

# Create the datasets
train_img8, train_lab = next(train_gen)
#test_img, test_lab = next(test_gen)

In [ ]:
train_img8.shape

In [ ]:
train_y = np.reshape(train_lab[:,0], (2239,1))
test_y = np.reshape(test_lab[:,0], (118,1))

In [ ]:
model5 = models.Sequential()
model5.add(layers.Conv2D(3, (3, 3), activation='relu',
                        input_shape=(8 ,8,  3)))
model5.add(layers.MaxPooling2D((2, 2)))

model5.add(layers.Flatten())
model5.add(layers.Dense(9, activation='softmax'))

model5.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [ ]:
history5 = model5.fit(train_img8,
                    train_y,
                    epochs=10,
                    batch_size=10, validation_split=0.3)

In [ ]:
df=model_results(history5, '8x8 5th model', df)
df

[Let me try to decrease the batch size already from the import, see if is changes anything.]

In [ ]:
"""# Directory path
train_data_dir = 'dataskin/Train'
test_data_dir = 'dataskin/Test'
#val_data_dir = 'dataset/validation_set'

# Get all the data in the directory data/validation (118 images), and reshape them
test_gen= ImageDataGenerator().flow_from_directory(
        test_data_dir, target_size=(8, 8), batch_size=5)

# Get all the data in the directory data/train (2239 images), and reshape them
train_gen = ImageDataGenerator().flow_from_directory(
        train_data_dir, target_size=(8, 8), batch_size=20)

# Create the datasets
train_img, train_lab = next(train_gen)
test_img, test_lab = next(test_gen)"""

In [ ]:
#train_y = np.reshape(train_lab[:,0], (20,1))
#test_y = np.reshape(test_lab[:,0], (5,1))

In [ ]:
#history6 = model5.fit(train_img,
#                    train_y,
#                    epochs=10,
#                    batch_size=10, validation_split=0.3)

In [ ]:
#df=model_results(history6, '8x8 5th model < batch on import', df)
#df

The good news is that even if the results change a lot in the first part where I do regualr NN, every time I run the code, they don't change so radically with the CNN.
I will keep track a bit but overall it seems like I can stop worrying about reproducibe results, if I use CNN right away.

## Starting to improve our model:

Taking from the german signs notebook

Nope. It's in AWS Sagemaker. But I can look back at it and use either a simple pipeline for loading and changing the size of the images, or for images augmentation to balance the classes of the different skin anomalies.
<br>A simple pipeline can even be "load, normalize, augment".

#### To improve our model we will take several steps:
   - normalize the data 
   - increase image resolution
   - add more epochs
   - add more layers
   - increase the batch size
   - change activation function and optimization
    
After we have done all this we will select the best performing model and we will also group up into 2 classes, cancerous and benign growth too.
<br>Maybe I should do that now actually.
<br>No I will do it after this first selection so I can say I have a good model to predict which of the 9 classes with this much accuracy.
<br>And then introduce the concept of two classes and decreasing recall.
<br>After that anyhow, do another round of tuning for the model, this time using:
 - drop out layers
 - regularization
 - normalization
 - data augmentation
 - balancing classes with data augmentation
 

### Improving the model with 9 classes:

Restart with an empty dataframe because while before we were looking for the lowest performance model now we are looking to improve it.

In [ ]:
df = pd.DataFrame()

Let us start by visualizing again the results from what we picked to be our naive model:

In [ ]:
df=model_results(history5, 'Naive', df)
df

#### Normalize the data:

To standardize the pixel values:

In [ ]:
train_img8 /= 255.

In [ ]:
train_img8

In [ ]:
history52 = model5.fit(train_img8,
                    train_y,
                    epochs=10,
                    batch_size=10, validation_split=0.3)

In [ ]:
df=model_results(history52, '8x8 5th model scaled pixels', df)
df

#### Increasing Image resolution:

In [ ]:
# Directory path
train_data_dir = 'dataskin/Train'
#test_data_dir = 'dataskin/Test'

# Get all the data in the directory data/train (2239 images), and reshape them
train_gen = ImageDataGenerator().flow_from_directory(
        train_data_dir, target_size=(64, 64), batch_size=2239)

# Create the datasets
train_img64, train_lab = next(train_gen)
#test_img64, test_lab = next(test_gen)

#normalizing again
train_img64 /= 255.
#test_img64 /= 255.

In [ ]:
train_y = np.reshape(train_lab[:,0], (2239,1))
test_y = np.reshape(test_lab[:,0], (118,1))

In [ ]:
model6 = models.Sequential()
model6.add(layers.Conv2D(3, (3, 3), activation='relu',
                        input_shape=(64, 64, 3)))
model6.add(layers.MaxPooling2D((2, 2)))

model6.add(layers.Flatten())
model6.add(layers.Dense(9, activation='softmax'))

model6.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [ ]:
history6b = model6.fit(train_img64,
                    train_y,
                    epochs=10,
                    batch_size=10, validation_split=0.3)

In [ ]:
df=model_results(history6b, '64x64 6th model', df)
df

Let us try increasing the size of the images imported further. Let us go all the way to the actually full size which is 256. These are already loaded at the beginning and are sotred in train_images_fs, test_images_fs.

In [ ]:
#normalizing again
train_images_fs /= 255.
test_images_fs /= 255.

In [ ]:
model6b = models.Sequential()
model6b.add(layers.Conv2D(3, (3, 3), activation='relu',
                        input_shape=(256, 256, 3)))
model6b.add(layers.MaxPooling2D((2, 2)))

model6b.add(layers.Flatten())
model6b.add(layers.Dense(9, activation='softmax'))

model6b.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [ ]:
history6c = model6b.fit(train_images_fs,
                    train_y,
                    epochs=10,
                    batch_size=10, validation_split=0.3)

In [ ]:
df=model_results(history6c, '256x256 6th model', df)
df

Let us see what happens with the full size.

#### Increasing number of epochs:

In [ ]:
history7 = model6.fit(train_img64,
                    train_y,
                    epochs=30,
                    batch_size=10, validation_split=0.3)

In [ ]:
df=model_results(history7, '64x64 6th model, 30 epochs', df)
df

#### Adding more layers:

In [ ]:
model7 = models.Sequential()
model7.add(layers.Conv2D(10, (3, 3), activation='relu',
                        input_shape=(64 ,64,  3)))
model7.add(layers.MaxPooling2D((2, 2)))

model7.add(layers.Conv2D(10, (4, 4), activation='relu'))
model7.add(layers.MaxPooling2D((2, 2)))

model7.add(layers.Conv2D(32, (3, 3), activation='relu'))
model7.add(layers.MaxPooling2D((2, 2)))

model7.add(layers.Flatten())
model7.add(layers.Dense(32, activation='relu'))
model7.add(layers.Dense(9, activation='softmax'))

model7.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [ ]:
history8 = model7.fit(train_img64,
                    train_y,
                    epochs=30,
                    batch_size=10, validation_split=0.3)

In [ ]:
df=model_results(history8, '64x64 7th model, 30 epochs, more layers', df)
df

#### Adding more layers and higher resolution images

In [ ]:
model7b = models.Sequential()
model7b.add(layers.Conv2D(10, (3, 3), activation='relu',
                        input_shape=(256 ,256,  3)))
model7b.add(layers.MaxPooling2D((2, 2)))

model7b.add(layers.Conv2D(10, (4, 4), activation='relu'))
model7b.add(layers.MaxPooling2D((2, 2)))

model7b.add(layers.Conv2D(32, (3, 3), activation='relu'))
model7b.add(layers.MaxPooling2D((2, 2)))

model7b.add(layers.Flatten())
model7b.add(layers.Dense(32, activation='relu'))
model7b.add(layers.Dense(9, activation='softmax'))

model7b.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [ ]:
#history8b = model7b.fit(train_images_fs,
#                    train_y,
#                    epochs=30,
#                    batch_size=10, validation_split=0.3)

In [ ]:
#df=model_results(history8b, '256x256 7th model', df)
#df

#### Increase the batch size: 50

In [ ]:
history9 = model7.fit(train_img64,
                    train_y,
                    epochs=30,
                    batch_size=50, validation_split=0.3)

In [ ]:
df=model_results(history9, '64x64 7th model, batch size=50', df)
df

The truth is that just getting one result for each of these attempts doesn't really tell us too much about the improvement or not of our model, because of the randomness of the results.
What would be more appropriate is for each change we do (in the size of the images, the number of epochs, the batch size) to try a few different values to be able to really get a sense of in which direction our model is moving with that change.
This would be very time consuming to do by hand, so the next step we are goign to take is building a grid search for these parameteres to get a better answer in terms of which ones are the optimal charateristic of our model, which gives us the best accuracy in determining the classification of the skin anomaly.

### Grid Search batch size and epochs:

Grid search is a model hyperparameter optimization technique.
In scikit learn this is a class, and when running a search with this class, we must input a dictionary of hyperparameters to evaluate in the param_grid argument. 
This dictionary contains the model parameter name and an array of values to try.

The GridSearchCV essentially builds and then evaluates one model for each combination of parameters. 
To evaluate each individual model, a 3-fold cross validation is used.

Even if we might have seen a better result with higher resolution images, we will keep the 64x64 ones for the grid search to limit the running time of our code.
Once we  find out from the GridSearch what are the best parameteres we can use those (running just one single model) with higher resolution images.

In [ ]:
# Function to create model, required for KerasClassifier
def create_model():
 # create model
    model = models.Sequential()
    model.add(layers.Conv2D(10, (3, 3), activation='relu',
                        input_shape=(64, 64, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Conv2D(10, (4, 4), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(9, activation='softmax'))

# Compile model
    model.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])
    return model

In [ ]:
#set X and Y
X=train_img64
Y=train_y

In [ ]:
# Use scikit-learn to grid search the batch size and epochs
# create model
model = KerasClassifier(model=create_model)
# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
best_params=grid_result.best_params_
best_params

In [ ]:
best_batch=grid_result.best_params_['batch_size']

In [ ]:
best_epochs=grid_result.best_params_['epochs']

Now we can run again a model with the higher resolution images, using the best parameters we obtained from the grid search.

In [ ]:
history10 = model7.fit(train_images64,
                    train_y,
                    epochs=best_epochs,
                    batch_size=best_batch, validation_split=0.3)

In [ ]:
df=model_results(history10, 'best model 1st Gridsearch', df)
df

### Grid Search Optimization Algorithm:

We can run again a GridSearch, but this time instead of trying different numbers of epochs and batch sizes, we will try different optimization algorithms. Here is a lists of them, and as we can see from the number of options, GridSearch is a good idea to shorten the time in trying all the different possible combinations.

#### Changing activation functions and optimizations:
Some of the options for activation functions are:
<br>relu
<br>swish
<br>softmax
<br>leaky relu
<br>sigmoid

<br>for optimizers:
<br>adam, sgd, 
<br>with optimizer I can also tweak the learning rate

<br>for loss functions:
categorical_crossentropy, 


We already have the function to create the model, and the X and Y we set before are teh same. We just need to change the grid for the search of the parameters.

In [ ]:
# create model
model = KerasClassifier(model=create_model, 
                        loss="sparse_categorical_crossentropy", 
                        epochs=best_epochs, batch_size=best_batch)
# define the grid search parameters
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
best_params=grid_result.best_params_
best_params

In [ ]:
best_opti=grid_result.best_params_['optimizer']

Creating a new model with the selected optimizer algorithm:

In [ ]:
 # create model
model8 = models.Sequential()
model8.add(layers.Conv2D(10, (3, 3), activation='relu',
                        input_shape=(64,64, 3)))
model8.add(layers.MaxPooling2D((2, 2)))
    
model8.add(layers.Conv2D(10, (4, 4), activation='relu'))
model8.add(layers.MaxPooling2D((2, 2)))

model8.add(layers.Conv2D(32, (3, 3), activation='relu'))
model8.add(layers.MaxPooling2D((2, 2)))

model8.add(layers.Flatten())
model8.add(layers.Dense(32, activation='relu'))
model8.add(layers.Dense(9, activation='softmax'))

# Compile model
model8.compile(loss='sparse_categorical_crossentropy',
              optimizer=best_opti,
              metrics=['acc'])

In [ ]:
history11 = model8.fit(train_img64,
                    train_y,
                    epochs=best_epochs,
                    batch_size=best_batch, validation_split=0.3)

In [ ]:
df=model_results(history11, 'best model Gridsearch optimization', df)
df

### Grid Search Learning Rate and Momentum

In [ ]:
# Need to add here

In [ ]:
# another grid search

In [ ]:
# for learning rate and momentum

We could also do a grid search to choose the best activation function, but it doesn't make much sense in our multiclass case because the most popular one is softmax for this case, there aren't really that many options. But we will do this type of grid search later when we will reduce our problem to a binary classification problem, and there will be more options of activation fucntions to use at that point.

What we can still tune though is the number of neurons in the hidden layers.
We will do this with another grid search.

### Grid Search number of neurons:

In [ ]:
# Function to create model, required for KerasClassifier
def create_model(neurons):
 # create model
    model = models.Sequential()
    model.add(layers.Conv2D(neurons, (3, 3), activation='relu',
                        input_shape=(64, 64, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Conv2D(neurons, (4, 4), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(neurons, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(9, activation='softmax'))

# Compile model
    model.compile(loss='sparse_categorical_crossentropy',
              optimizer=best_opti,
              metrics=['acc'])
    return model

In [ ]:
# fix random seed for reproducibility
seed = 7
tf.random.set_seed(seed)
#set X and Y
X=train_img64
Y=train_y

In [ ]:
# create model
model = KerasClassifier(model=create_model, epochs=best_epochs, batch_size=best_batch)
# define the grid search parameters
neurons = [5, 10, 20, 30]
param_grid = dict(model__neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
neurons=grid_result.best_params_['model__neurons']

In [ ]:
model9 = models.Sequential()
model9.add(layers.Conv2D(neurons, (3, 3), activation='relu',
                        input_shape=(64, 64, 3)))
model9.add(layers.MaxPooling2D((2, 2)))
    
model9.add(layers.Conv2D(neurons, (4, 4), activation='relu'))
model9.add(layers.MaxPooling2D((2, 2)))
model9.add(layers.Conv2D(neurons, (3, 3), activation='relu'))
model9.add(layers.MaxPooling2D((2, 2)))
model9.add(layers.Flatten())
model9.add(layers.Dense(32, activation='relu'))
model9.add(layers.Dense(9, activation='softmax'))

# Compile model
model9.compile(loss='sparse_categorical_crossentropy',
              optimizer=best_opti,
              metrics=['acc'])

In [ ]:
history12 = model9.fit(train_img64,
                    train_y,
                    epochs=best_epochs,
                    batch_size=best_batch, validation_split=0.3)

In [ ]:
df=model_results(history11, 'best model Gridsearch neurons', df)
df

## Tuning

### Regularization

In [ ]:
from tensorflow.keras import regularizers

model9 = models.Sequential()
model9.add(layers.Conv2D(neurons, (3, 3), activation='relu',
                        input_shape=(64, 64, 3)))
model9.add(layers.MaxPooling2D((2, 2)))
    
model9.add(layers.Conv2D(neurons, (4, 4), activation='relu',
                         kernel_regularizer=regularizers.L2(l2=0.05)))
model9.add(layers.MaxPooling2D((2, 2)))
model9.add(layers.Conv2D(neurons, (3, 3), activation='relu', 
                        kernel_regularizer=regularizers.L2(l2=0.05)))
model9.add(layers.MaxPooling2D((2, 2)))
model9.add(layers.Flatten())
model9.add(layers.Dense(32, activation='relu', 
                       kernel_regularizer=regularizers.L2(l2=0.05)))
model9.add(layers.Dense(9, activation='softmax'))

# Compile model
model9.compile(loss='sparse_categorical_crossentropy',
              optimizer=best_opti,
              metrics=['acc'])

In [ ]:
history13 = model9.fit(train_img64,
                    train_y,
                    epochs=best_epochs,
                    batch_size=best_batch, validation_split=0.3)

In [ ]:
df=model_results(history11, 'after regularization', df)
df

>Look into L1 and see if it makes sense to do L1 regularization too.
>Play also around a bit and find the best parameters for L1 and L2.

### Dropout Regularization

>Check out grid search blogpost

### Early Stopping

#### Best model

#### Testing on the test

## Changing to two classes cancerous and bening

In [ ]:
# Work on the data

In [ ]:
# If there is an imbalance re balance the classes with augmentation

In [ ]:
# Create function to extract recall

In [ ]:
y_hat_train=model.predict(train_img)
y_hat_test=model.predict(test_img)

Clearly here I have to round to get the actual prediction for the class.

In [ ]:
pr_train=precision_score(train_labels, train_labels, average=None)
#rec_train=recall_score(train_labels, y_hat_train)
#acc_train=accuracy_score(train_labels, y_hat_train)
#f1_train=f1_score(train_labels, y_hat_train)

#pr_test=precision_score(test_labels, y_hat_test)
#rec_test=recall_score(test_labels, y_hat_test)
#acc_test=accuracy_score(test_labels, y_hat_test)
#f1_test=f1_score(test_labels, y_hat_test)
pr_train
#print("Train: \nprecision", pr_train,"\nrecall:", rec_train, '\naccuracy:', acc_train, '\nf1 score:', f1_train,'\n')
#print("Test: \nprecision", pr_test,"\nrecall:", rec_test, '\naccuracy:', acc_test, '\nf1 score:', f1_test)

In [ ]:
# Create function to compare the models (similar to previous one but comparing recall)

### Re tuning of the parameters
With this time binary classification problem

In [ ]:
# Grid Searches

In [ ]:
# Tuning with normalization and regularization

In [ ]:
# Best model

In [ ]:
# Testing on the test set

## LIME